Una  ves emos echo la limpieza de los datos provenientes de YELP , Google y Overpass procederemos hacer un merge entre los datos para simplificar los archivos.

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.5 MB/s eta 0:00:00


In [ ]:
#Librerias
import pandas as pd
import unidecode
import os
import random
import string

 Realizamos un merge con los datos  del servicio de entretenimiento obtenidos de Overpass

In [ ]:
# Listamos los  archivos que se encuentran separados por años a combinar
files = [
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2014.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2015.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2016.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2017.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2018.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2019.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2020.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2021.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2022.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2023.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/Parques_de_diversiones_Centros_comerciales csv/Parques_de_diversiones_Centros_comerciales_2024.csv'
]

dfs = []
for file in files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        dfs.append(df)
    else:
        print(f"El archivo no existe: {file}")

# Realisamos un merge con los archivos listados
merged_df = pd.concat(dfs, ignore_index=True)

# Guardamos el archivo
output_path = '/content/sample_data/Parques_de_diversiones_Centros_comerciales_combined.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo combinado guardado en: {output_path}")


Archivo combinado guardado en: /content/sample_data/Parques_de_diversiones_Centros_comerciales_combined.csv


Al archivo obtenido anteriormente mediante el archivos dimensiones procedamos a completar  los datos de condado  y id del establecimiento

In [ ]:
# Cargamos los archivos de entrada
combined_file = '/content/sample_data/Parques_de_diversiones_Centros_comerciales_combined.csv'
ciudades_dim_file = '/content/sample_data/ciudades_dim.csv'
centros_entretenimiento_dim_file = '/content/drive/MyDrive/categorias3.3/centros_entretenimiento_dim.csv'

combined_df = pd.read_csv(combined_file)
ciudades_dim = pd.read_csv(ciudades_dim_file)
centros_entretenimiento_dim = pd.read_csv(centros_entretenimiento_dim_file)

#Nos asegurar que las columnas ids sean de tipo string
ciudades_dim['id_ciudad'] = ciudades_dim['id_ciudad'].astype(str)
ciudades_dim['id_condado'] = ciudades_dim['id_condado'].astype(str)
combined_df['id_ciudad'] = combined_df['id_ciudad'].astype(str)
centros_entretenimiento_dim['id_centros_entretenimiento'] = centros_entretenimiento_dim['id_centros_entretenimiento'].astype(str)

# Cambiamos el nombre de la columna 'nombre' a 'centros_entretenimiento'
if 'nombre' in combined_df.columns:
    combined_df.rename(columns={'nombre': 'centros_entretenimiento'}, inplace=True)

# Normalizamos las columnas con nombres para evitar signos o letras en mayusculas
def normalizar_nombre(nombre):
    if pd.isnull(nombre):
        return None
    return unidecode.unidecode(str(nombre)).lower().strip()

combined_df['ciudad_norm'] = combined_df['ciudad'].apply(normalizar_nombre)
ciudades_dim['ciudad_norm'] = ciudades_dim['ciudad'].apply(normalizar_nombre)
combined_df['centros_entretenimiento_normalizado'] = combined_df['centros_entretenimiento'].apply(normalizar_nombre)
centros_entretenimiento_dim['centros_entretenimiento_normalizado'] = centros_entretenimiento_dim['centros_entretenimiento'].apply(normalizar_nombre)

# Realizamos el merge para completar datos faltantes
merged_df = combined_df.merge(
    ciudades_dim[['ciudad_norm', 'id_ciudad', 'id_condado', 'condado',
                  'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
                  'codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad']],
    on='ciudad_norm',
    how='left',
    suffixes=('', '_ciudad_dim')
)

if 'id_ciudad' not in merged_df.columns:
    print("Error: La columna 'id_ciudad' no se encontró después del merge.")
else:
    print("La columna 'id_ciudad' se asignó correctamente.")

missing_id_ciudad = merged_df['id_ciudad'].isnull().sum()
if missing_id_ciudad > 0:
    print(f"Registros sin id_ciudad: {missing_id_ciudad}")
    merged_df[merged_df['id_ciudad'].isnull()].to_csv('/content/sample_data/registros_sin_id_ciudad.csv', index=False)
    print("Registros sin id_ciudad guardados en: /content/sample_data/registros_sin_id_ciudad.csv")

# Asignamos id a  la columna centros de entretenimiento y agregamos nuevos nombres si no existen en el archivo dim
def generar_id_unico(existentes):
    while True:
        nuevo_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
        if nuevo_id not in existentes:
            return nuevo_id

ids_existentes = set(centros_entretenimiento_dim['id_centros_entretenimiento'])
nuevas_unidades = []

for index, row in merged_df.iterrows():
    nombre_norm = row['centros_entretenimiento_normalizado']
    matching_row = centros_entretenimiento_dim[centros_entretenimiento_dim['centros_entretenimiento_normalizado'] == nombre_norm]

    if not matching_row.empty:
        merged_df.at[index, 'id_centros_entretenimiento'] = matching_row['id_centros_entretenimiento'].values[0]
    else:
        nuevo_id = generar_id_unico(ids_existentes)
        ids_existentes.add(nuevo_id)
        merged_df.at[index, 'id_centros_entretenimiento'] = nuevo_id
        nuevas_unidades.append({
            'id_centros_entretenimiento': nuevo_id,
            'centros_entretenimiento': row['centros_entretenimiento'],
            'centros_entretenimiento_normalizado': nombre_norm
        })

# Agregamos nuevos datos al archivo dim
if nuevas_unidades:
    nuevas_unidades_df = pd.DataFrame(nuevas_unidades)
    centros_entretenimiento_dim = pd.concat([centros_entretenimiento_dim, nuevas_unidades_df], ignore_index=True)
    centros_entretenimiento_dim.drop(columns=['centros_entretenimiento_normalizado'], inplace=True)
    centros_entretenimiento_dim.to_csv(centros_entretenimiento_dim_file, index=False)
    print(f"Se han añadido {len(nuevas_unidades)} nuevos centros de entretenimiento al archivo de referencia.")

# organisamos las columnas
ordered_columns = [
    'id_centros_entretenimiento', 'centros_entretenimiento', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad', 'codigo_postal_ciudad',
    'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio', 'enlaces_google_maps', 'anio'
]

for col in ordered_columns:
    if col not in merged_df.columns:
        merged_df[col] = None

merged_df = merged_df[ordered_columns]

# Guardamos el archivo procesado final
output_path = '/content/sample_data/Parques_de_diversiones_Centros_comerciales_final.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo procesado y guardado en: {output_path}")


La columna 'id_ciudad' se asignó correctamente.
Archivo procesado y guardado en: /content/sample_data/Parques_de_diversiones_Centros_comerciales_final.csv


Cambiamos los nombres de columnas para que queden iguales en todos los archivos

In [ ]:
# Ruta del archivo de entrada y salida
input_file = '/content/drive/MyDrive/categorias10/servicios_entretenimiento.csv'
output_file = '/content/drive/MyDrive/categorias10/servicios_entretenimiento_coregido.csv'

# Cargar el archivo
df = pd.read_csv(input_file)

# Renombrar las columnas según lo solicitado
column_renames = {
    'atributo': 'atributos',
    'categoria': 'categorias',
    '.url': 'url_del_negocio',
    'ranquing_por_usuario': 'puntuacion_usuarios'

}

df.rename(columns=column_renames, inplace=True)

# Guardar el archivo actualizado
df.to_csv(output_file, index=False)

print(f"Archivo actualizado guardado en: {output_file}")


Archivo actualizado guardado en: /content/drive/MyDrive/categorias10/servicios_entretenimiento_coregido.csv


Unificamos los archivos obtenidos  de Google Yelp y Overpass

In [ ]:
#Cargamos la ruta de los archivos que vamos a usar en el merge

file1 = '/content/sample_data/Parques_de_diversiones_Centros_comerciales_final.csv'
file2 = '/content/drive/MyDrive/categorias3.3/entretenimiento_ordenado.csv'
file3 = '/content/drive/MyDrive/categorias10/servicios_entretenimiento_coregido.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

# Agregamos una columna de categoría general "Entretenimiento" en cada archivo

df1['categoria_general'] = 'Entretenimiento'
df2['categoria_general'] = 'Entretenimiento'
df3['categoria_general'] = 'Entretenimiento'

# Realizamos el merge de los tres archivos

merged_df = pd.concat([df1, df2, df3], ignore_index=True)
print("Columnas disponibles después del merge:", merged_df.columns)

if {'centros_entretenimiento', 'anio', 'latitud_ciudad'}.issubset(merged_df.columns):
    merged_df['non_null_count'] = merged_df.notna().sum(axis=1)
    merged_df = merged_df.sort_values(by='non_null_count', ascending=False)
    merged_df = merged_df.drop_duplicates(subset=['centros_entretenimiento', 'anio', 'latitud_ciudad'], keep='first')
    merged_df = merged_df.drop(columns=['non_null_count'], errors='ignore')  #
else:
    print("Las columnas necesarias para resolver duplicados no están presentes en el DataFrame.")

# Ordenamos los datos por la columna 'anio' de menor a mayor
if 'anio' in merged_df.columns:
    merged_df = merged_df.sort_values(by='anio', ascending=True)

# Guardamos  el resultado
output_path = '/content/sample_data/servicios_entretenimiento_combinado.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo combinado con categoría general 'Entretenimiento', sin duplicados y ordenado guardado en: {output_path}")




Columnas disponibles después del merge: Index(['id_centros_entretenimiento', 'centros_entretenimiento', 'direccion',
       'id_condado', 'condado', 'codigo_postal_condado', 'latitud_condado',
       'longitud_condado', 'id_ciudad', 'ciudad', 'codigo_postal_ciudad',
       'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos',
       'categorias', 'puntuacion_usuarios', 'numero_de_reviews',
       'analisis_sentimientos', 'url_del_negocio', 'enlaces_google_maps',
       'anio', 'categoria_general', 'url'],
      dtype='object')
Archivo combinado con categoría general 'Entretenimiento', sin duplicados y ordenado guardado en: /content/sample_data/servicios_entretenimiento_combinado.csv


Eliminamos signos inesesarios

In [ ]:
import pandas as pd

# Archivo de entrada y salida
input_file = '/content/sample_data/servicios_entretenimiento_combinado.csv'
output_file = '/content/sample_data/servicios_entretenimiento_combinado_formateado.csv'

# Cargar el archivo
df = pd.read_csv(input_file)

# Normalizar nombres de columnas (sin transformar "&")
df.columns = [
    col_name.strip()
    .replace(" ", "_")
    .replace("& ", "and")
    .replace(",", "")
    .replace("(", "")
    .replace(")", "")
    .replace("$", "")
    .replace(";", "")
    .replace("=", "")
    .replace(".", "_")
    .replace("\n", "")
    .replace("\t", "")
    for col_name in df.columns
]

# Asignar tipos de datos a las columnas según lo solicitado
column_types = {
    'id_centros_entretenimiento': 'string',
    'centros_entretenimiento': 'string',
    'direccion': 'string',
    'id_condado': 'string',
    'condado': 'string',
    'codigo_postal_condado': 'string',
    'latitud_condado': 'float',
    'longitud_condado': 'float',
    'id_ciudad': 'string',
    'ciudad': 'string',
    'codigo_postal_ciudad': 'string',
    'latitud_ciudad': 'float',
    'longitud_ciudad': 'float',
    'estado': 'string',
    'categorias': 'string',
    'puntuacion_usuarios': 'float',
    'analisis_sentimientos': 'string',
    'url_del_negocio': 'string',
    'enlaces_google_maps': 'string',
    'anio': 'int'
}

# Aplicar tipos de datos
for column, dtype in column_types.items():
    if column in df.columns:
        df[column] = df[column].astype(dtype)

# Eliminar caracteres no deseados en todas las columnas de tipo string
columns_to_clean = df.select_dtypes(include=['string']).columns
df[columns_to_clean] = df[columns_to_clean].replace(
    to_replace=r"[\[\]\{\}'\"]", value="", regex=True
)


# Guardar el archivo procesado
df.to_csv(output_file, index=False)

print(f"Archivo procesado y guardado en: {output_file}")




Archivo procesado y guardado en: /content/sample_data/servicios_entretenimiento_combinado_formateado.csv


Realizamos un merge con los datos  del servicio de entretenimiento obtenidos de Overpass

In [ ]:
# Listamos los archivos a combinar
files = [
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2014.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2015.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2016.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2017.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2018.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2019.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2020.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2021.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2022.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2023.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2044.csv',
]

dfs = []
for file in files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        dfs.append(df)
    else:
        print(f"El archivo no existe: {file}")

# Concatenamos todos los DataFrames
merged_df = pd.concat(dfs, ignore_index=True)

# Guardamos el archivo combinado
output_path = '/content/unidad_educativa_combined.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo combinado guardado en: {output_path}")


El archivo no existe: /content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/unidad educativa csv/unidad_educativa_2044.csv
Archivo combinado guardado en: /content/unidad_educativa_combined.csv


Nos aseguramos que loas columnas tegan en el archivo combinado y el archivo dim  ,los mismosnombres

In [ ]:
# Cargamos la ruta del archivo dim
centros_educacion_dim_file = '/content/drive/MyDrive/categorias3.3/centros_educacion_dim.csv'
centros_educacion_dim = pd.read_csv(centros_educacion_dim_file)

# Renombramos  las columnas
centros_educacion_dim.rename(columns={
    'id_centros_educativos': 'id_centros_educacion',
    'centros_educativos': 'centros_educacion'
}, inplace=True)

# Guardamos los cambios
centros_educacion_dim.to_csv(centros_educacion_dim_file, index=False)

print(f"Archivo actualizado y guardado en: {centros_educacion_dim_file}")


Archivo actualizado y guardado en: /content/drive/MyDrive/categorias3.3/centros_educacion_dim.csv


In [ ]:
# Ruta del archivo de entrada y salida
input_file = '/content/drive/MyDrive/categorias10/servicios_educacion.csv'
output_file = '/content/drive/MyDrive/categorias10/servicios_educacion_coregido.csv'

# Cargar el archivo
df = pd.read_csv(input_file)

# Renombrar las columnas según lo solicitado
column_renames = {
    'atributo': 'atributos',
    'categoria': 'categorias',
    '.url': 'url_del_negocio',
    'ranquing_por_usuario': 'puntuacion_usuarios'

}

df.rename(columns=column_renames, inplace=True)

# Guardar el archivo actualizado
df.to_csv(output_file, index=False)

print(f"Archivo actualizado guardado en: {output_file}")


Archivo actualizado guardado en: /content/drive/MyDrive/categorias10/servicios_educacion_coregido.csv


Completamos archivos faltantes mediante los archivos de dimenciones

In [ ]:
# Cargamos los archivos nesesrios para el merge
combined_file = '/content/unidad_educativa_combined.csv'
ciudades_dim_file = '/content/sample_data/ciudades_dim.csv'
centros_educacion_dim_file = '/content/drive/MyDrive/categorias3.3/centros_educacion_dim.csv'

combined_df = pd.read_csv(combined_file)
ciudades_dim = pd.read_csv(ciudades_dim_file)
centros_educacion_dim = pd.read_csv(centros_educacion_dim_file)

# Revisamos que el formato sea igual en las columnas nesesarias

ciudades_dim['id_ciudad'] = ciudades_dim['id_ciudad'].astype(str)
ciudades_dim['id_condado'] = ciudades_dim['id_condado'].astype(str)
combined_df['id_ciudad'] = combined_df['id_ciudad'].astype(str)
centros_educacion_dim['id_centros_educacion'] = centros_educacion_dim['id_centros_educacion'].astype(str)

# Cambiamos  el nombre de la columna 'nombre' a 'centros_educacion' si existe
if 'nombre' in combined_df.columns:
    combined_df.rename(columns={'nombre': 'centros_educacion'}, inplace=True)

# Normalizamos las columnas a usar
def normalizar_nombre(nombre):
    if pd.isnull(nombre):
        return None
    return unidecode.unidecode(str(nombre)).lower().strip()

combined_df['ciudad_norm'] = combined_df['ciudad'].apply(normalizar_nombre)
ciudades_dim['ciudad_norm'] = ciudades_dim['ciudad'].apply(normalizar_nombre)
combined_df['centros_educacion_normalizado'] = combined_df['centros_educacion'].apply(normalizar_nombre)
centros_educacion_dim['centros_educacion_normalizado'] = centros_educacion_dim['centros_educacion'].apply(normalizar_nombre)

# Realizamos el merge para completar la información de la ciudad
merged_df = combined_df.merge(
    ciudades_dim[['ciudad_norm', 'id_ciudad', 'id_condado', 'condado',
                  'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
                  'codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad']],
    on='ciudad_norm',
    how='left',
    suffixes=('', '_ciudad_dim')
)

if 'id_ciudad' not in merged_df.columns:
    print("Error: La columna 'id_ciudad' no se encontró después del merge.")
else:
    print("La columna 'id_ciudad' se asignó correctamente.")

missing_id_ciudad = merged_df['id_ciudad'].isnull().sum()
if missing_id_ciudad > 0:
    print(f"Registros sin id_ciudad: {missing_id_ciudad}")
    merged_df[merged_df['id_ciudad'].isnull()].to_csv('/content/sample_data/registros_sin_id_ciudad.csv', index=False)
    print("Registros sin id_ciudad guardados en: /content/sample_data/registros_sin_id_ciudad.csv")

# Asignamos las ids a centros de educacion y agregar nuevos nombres si no existen en el archivo dim
def generar_id_unico(existentes):
    while True:
        nuevo_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
        if nuevo_id not in existentes:
            return nuevo_id

ids_existentes = set(centros_educacion_dim['id_centros_educacion'])
nuevas_unidades = []

for index, row in merged_df.iterrows():
    nombre_norm = row['centros_educacion_normalizado']
    matching_row = centros_educacion_dim[centros_educacion_dim['centros_educacion_normalizado'] == nombre_norm]

    if not matching_row.empty:
        merged_df.at[index, 'id_centros_educacion'] = matching_row['id_centros_educacion'].values[0]
    else:
        nuevo_id = generar_id_unico(ids_existentes)
        ids_existentes.add(nuevo_id)
        merged_df.at[index, 'id_centros_educacion'] = nuevo_id
        nuevas_unidades.append({
            'id_centros_educacion': nuevo_id,
            'centros_educacion': row['centros_educacion'],
            'centros_educacion_normalizado': nombre_norm
        })

if nuevas_unidades:
    nuevas_unidades_df = pd.DataFrame(nuevas_unidades)
    centros_educacion_dim = pd.concat([centros_educacion_dim, nuevas_unidades_df], ignore_index=True)
    centros_educacion_dim.drop(columns=['centros_educacion_normalizado'], inplace=True)
    centros_educacion_dim.to_csv(centros_educacion_dim_file, index=False)
    print(f"Se han añadido {len(nuevas_unidades)} nuevos centros de educación al archivo de referencia.")

# ordenamos las columnas
ordered_columns = [
    'id_centros_educacion', 'centros_educacion', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad', 'codigo_postal_ciudad',
    'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio', 'enlaces_google_maps', 'anio'
]
for col in ordered_columns:
    if col not in merged_df.columns:
        merged_df[col] = None

merged_df = merged_df[ordered_columns]

# Guardamos el archivo procesado
output_path = '/content/sample_data/servicios_educacion2_final.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo procesado y guardado en: {output_path}")





La columna 'id_ciudad' se asignó correctamente.
Archivo procesado y guardado en: /content/sample_data/servicios_educacion2_final.csv


Eliminamos en la columna ciudad el nombre de Dania ya que la ciudad no esta definida

In [ ]:
# Cargamos el archivo
file_path = '/content/sample_data/servicios_educacion2_final.csv'
data = pd.read_csv(file_path)

filtered_data = data[data['ciudad'] != 'Dania']

# Guardamos el resultado
output_path = '/content/sample_data/servicios_educacion2_filtered.csv'
filtered_data.to_csv(output_path, index=False)

output_path


'/content/sample_data/servicios_educacion2_filtered.csv'

Unificamos los archivos obtenidos  de Google Yelp y Overpass

In [ ]:
# Cargamos los archivos nesesarios para el merge
file1 = '/content/sample_data/servicios_educacion2_filtered.csv'
file2 = '/content/drive/MyDrive/categorias3.3/educacion_ordenado.csv'
file3 = '/content/drive/MyDrive/categorias10/servicios_educacion_coregido.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

# Agregamos una columna de categoría general "educacion"
df1['categoria_general'] = 'educacion'
df2['categoria_general'] = 'educacion'
df3['categoria_general'] = 'educacion'

required_columns = [
    'id_centros_educacion', 'centros_educacion', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad',
    'codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos',
    'categorias', 'puntuacion_usuarios', 'numero_de_reviews', 'analisis_sentimientos',
    'url_del_negocio', 'enlaces_google_maps', 'anio', 'categoria_general'
]
for df in [df1, df2, df3]:
    for col in required_columns:
        if col not in df.columns:
            df[col] = pd.NA

# Realisamos el merge
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

# eliminamos filas duplicadas

if {'centros_educacion', 'anio', 'latitud_ciudad'}.issubset(merged_df.columns):
    merged_df['non_null_count'] = merged_df.notna().sum(axis=1)
    merged_df = merged_df.sort_values(by='non_null_count', ascending=False)
    merged_df = merged_df.drop_duplicates(subset=['centros_educacion', 'anio', 'latitud_ciudad'], keep='first')
    merged_df = merged_df.drop(columns=['non_null_count'], errors='ignore')

# Ordenamos los datos por la columna 'anio'
if 'anio' in merged_df.columns:
    merged_df = merged_df.sort_values(by='anio', ascending=True)

# Guardamos el resultado
output_path = '/content/sample_data/servicios_educacion_combinado2.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo combinado con categoría general 'Educacion', sin duplicados y ordenado guardado en: {output_path}")




Archivo combinado con categoría general 'Educacion', sin duplicados y ordenado guardado en: /content/sample_data/servicios_educacion_combinado2.csv


Ordenamos las columnas

In [ ]:
# Definimos el oreden de las columnas
columns_to_keep = [
    'id_centros_educacion', 'centros_educacion', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad',
    'codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos',
    'categorias', 'puntuacion_usuarios', 'numero_de_reviews', 'analisis_sentimientos',
    'url_del_negocio', 'enlaces_google_maps', 'categoria_general', 'anio'
]


file_path = '/content/sample_data/servicios_educacion2_final.csv'
data = pd.read_csv(file_path)

if 'categoria_general' not in data.columns:
    data['categoria_general'] = 'educacion'

filtered_data = data[data['ciudad'] != 'Dania']

final_data = filtered_data[columns_to_keep]

# Guardamos el resultado
output_path = '/content/sample_data/servicios_educacion2_final_filtered.csv'
final_data.to_csv(output_path, index=False)

print(f"Archivo procesado guardado en: {output_path}")


Archivo procesado guardado en: /content/sample_data/servicios_educacion2_final_filtered.csv


Eliminamos los caracteres imnesesarios

In [ ]:
#Cargamos los archivos de  entrada y salida
input_file = '/content/sample_data/servicios_educacion2_final_filtered.csv'
output_file = '/content/sample_data/servicios_educacion2_final_filtered2.csv.csv'

df = pd.read_csv(input_file)

# Normalizamos nombres de columnas
df.columns = [
    col_name.strip()
    .replace(" ", "_")
    .replace(",", "")
    .replace("(", "")
    .replace(")", "")
    .replace("$", "")
    .replace(";", "")
    .replace("=", "")
    .replace(".", "_")
    .replace("\n", "")
    .replace("\t", "")
    for col_name in df.columns
]

# Asignamos el formato de los tipos de datos a las columnas
column_types = {
    'id_centros_entretenimiento': 'string',
    'centros_entretenimiento': 'string',
    'direccion': 'string',
    'id_condado': 'string',
    'condado': 'string',
    'codigo_postal_condado': 'string',
    'latitud_condado': 'float',
    'longitud_condado': 'float',
    'id_ciudad': 'string',
    'ciudad': 'string',
    'codigo_postal_ciudad': 'string',
    'latitud_ciudad': 'float',
    'longitud_ciudad': 'float',
    'estado': 'string',
    'categorias': 'string',
    'puntuacion_usuarios': 'float',
    'analisis_sentimientos': 'string',
    'url_del_negocio': 'string',
    'enlaces_google_maps': 'string',
    'anio': 'int'
}

for column, dtype in column_types.items():
    if column in df.columns:
        df[column] = df[column].astype(dtype)

# Eliminamos caracteres no deseados en todas las columnas de tipo string
columns_to_clean = df.select_dtypes(include=['string']).columns
df[columns_to_clean] = df[columns_to_clean].replace(
    to_replace=r"[\[\]\{\}'\"]", value="", regex=True
)


# Guardamos el archivo procesado
df.to_csv(output_file, index=False)

print(f"Archivo procesado y guardado en: {output_file}")

Archivo procesado y guardado en: /content/sample_data/servicios_educacion2_final_filtered2.csv.csv


Cambiamos los nombres de colunas en los archivos dim y servicios restaurantes

In [ ]:
# Cargamos la ruta del archivo
file_path = '/content/drive/MyDrive/categorias3.3/servicios_restaurantes_dim.csv'
o
df = pd.read_csv(file_path)

df.rename(columns={'id_centros_restaurantes': 'id_servicios_restaurantes',
                   'centros_restaurantes': 'servicios_restaurantes'}, inplace=True)

# Guardamos el archivo actualizado
df.to_csv(file_path, index=False)

"Las columnas han sido renombradas y el archivo actualizado correctamente."


'Las columnas han sido renombradas y el archivo actualizado correctamente.'

 Realizamos un merge con los datos  del servicio restaurantes  obtenidos de Overpass

In [ ]:
#Cargamos archivos de entrada
files = [
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/restaurantes csv/restaurantes_2023.csv',
    '/content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/restaurantes csv/restaurant_turvo_2022.csv'
]

dfs = []

for file in files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        print(f"Columnas originales en {file}: {list(df.columns)}")

        if 'nombre' in df.columns:
            df.rename(columns={'nombre': 'servicio_restaurantes'}, inplace=True)
        else:
            print(f"La columna 'nombre' no existe en el archivo: {file}. Se omite este archivo.")
            continue

        if 'id_nombre' in df.columns:
            df.drop(columns=['id_nombre'], inplace=True)

        dfs.append(df)
    else:
        print(f"El archivo no existe: {file}")

if len(dfs) == 0:
    print("No se encontraron archivos válidos para combinar.")
    exit()

# Realizamos el merge
merged_df = pd.concat(dfs, ignore_index=True)
print(f"Columnas después de concatenar: {list(merged_df.columns)}")

# Cargamos la ruta del archivo de servicios_restaurantes_dim y otorgamos las id correspondientes
servicios_dim_file = '/content/drive/MyDrive/categorias3.3/servicios_restaurantes_dim.csv'
if os.path.exists(servicios_dim_file):
    servicios_dim = pd.read_csv(servicios_dim_file)
    print(f"Columnas en servicios_restaurantes_dim: {list(servicios_dim.columns)}")
    servicios_dim.rename(columns={'servicios_restaurantes': 'servicio_restaurantes'}, inplace=True)

    servicios_dict = dict(zip(servicios_dim['servicio_restaurantes'], servicios_dim['id_servicios_restaurantes']))

    merged_df['id_servicio_restaurantes'] = merged_df['servicio_restaurantes'].map(servicios_dict)

    for index, row in merged_df[merged_df['id_servicio_restaurantes'].isnull()].iterrows():
        new_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
        servicios_dict[row['servicio_restaurantes']] = new_id
        merged_df.at[index, 'id_servicio_restaurantes'] = new_id

    nuevos_servicios = merged_df[~merged_df['servicio_restaurantes'].isin(servicios_dim['servicio_restaurantes'])]
    if not nuevos_servicios.empty:
        nuevos_registros = nuevos_servicios[['servicio_restaurantes', 'id_servicio_restaurantes']].drop_duplicates()
        nuevos_registros.to_csv(servicios_dim_file, mode='a', index=False, header=False)
else:
    print(f"El archivo de servicios no existe: {servicios_dim_file}")
    exit()

# Guardamos el archivo combinado
output_path = '/content/restaurantes_combined.csv'
merged_df.to_csv(output_path, index=False)
print(f"Archivo combinado guardado en: {output_path}")






Columnas originales en /content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/restaurantes csv/restaurantes_2023.csv: ['id_nombre', 'nombre', 'direccion', 'id_ciudad', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud', 'anio']
Columnas originales en /content/drive/MyDrive/data/Datos Procesados-20250109T140622Z-001/Datos Procesados/restaurantes csv/restaurant_turvo_2022.csv: ['id_nombre', 'nombre', 'direccion', 'id_ciudad', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud', 'anio']
Columnas después de concatenar: ['servicio_restaurantes', 'direccion', 'id_ciudad', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud', 'anio']
Columnas en servicios_restaurantes_dim: ['id_servicios_restaurantes', 'servicios_restaurantes']
Archivo combinado guardado en: /content/restaurantes_combined.csv


ponemos los datos de condado

In [ ]:
#Cargamos las rutas de los archivos
combined_file = '/content/restaurantes_combined.csv'
ciudades_dim_file = '/content/sample_data/ciudades_dim.csv'
combined_df = pd.read_csv(combined_file)
ciudades_dim = pd.read_csv(ciudades_dim_file)

# Renombramos las columnas en combined_df
combined_df.rename(columns={
    'latitud': 'latitud_ciudad',
    'longitud': 'longitud_ciudad',
    'codigo_postal': 'codigo_postal_ciudad'
}, inplace=True)

# Normalizamos las claves en ambos DataFrames
for col in ['id_ciudad', 'ciudad', 'codigo_postal_ciudad']:
    combined_df[col] = combined_df[col].astype(str).str.strip().str.lower()
    ciudades_dim[col] = ciudades_dim[col].astype(str).str.strip().str.lower()
ciudades_dim = ciudades_dim.assign(
    codigo_postal_ciudad=ciudades_dim['codigo_postal_ciudad'].str.split(',')
).explode('codigo_postal_ciudad').reset_index(drop=True)

ciudades_dim['codigo_postal_ciudad'] = ciudades_dim['codigo_postal_ciudad'].str.strip().str.lower()

print("\nClaves únicas en combined_df:")
print(combined_df[['id_ciudad', 'ciudad', 'codigo_postal_ciudad']].drop_duplicates().head(10))

print("\nClaves únicas en ciudades_dim:")
print(ciudades_dim[['id_ciudad', 'ciudad', 'codigo_postal_ciudad']].drop_duplicates().head(10))

merged_df = combined_df.merge(
    ciudades_dim[['id_condado', 'condado', 'codigo_postal_condado', 'latitud_condado', 'longitud_condado',
                  'id_ciudad', 'ciudad', 'codigo_postal_ciudad']],
    on=['id_ciudad', 'ciudad', 'codigo_postal_ciudad'],
    how='left'
)
print("\nDatos faltantes en columnas relacionadas con el condado:")
print(merged_df[['id_condado', 'condado', 'codigo_postal_condado', 'latitud_condado', 'longitud_condado']].isnull().sum())

unmatched_rows = merged_df[merged_df['id_condado'].isnull()]
print("\nFilas sin coincidencia (primeras 10):")
print(unmatched_rows[['id_ciudad', 'ciudad', 'codigo_postal_ciudad']].head(10))

unmatched_rows.to_csv('/content/unmatched_rows.csv', index=False)
print("\nFilas sin coincidencias guardadas en: /content/unmatched_rows.csv")

if 'id_servicios_restaurantes' not in merged_df.columns:
    merged_df['id_servicios_restaurantes'] = combined_df['id_servicio_restaurantes']
if 'servicios_restaurantes' not in merged_df.columns:
    merged_df['servicios_restaurantes'] = combined_df['servicio_restaurantes']

# Ordenamos las columnas
ordered_columns = [
    'id_servicios_restaurantes', 'servicios_restaurantes', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad', 'codigo_postal_ciudad',
    'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio', 'enlaces_google_maps', 'anio'
]

for col in ordered_columns:
    if col not in merged_df.columns:
        merged_df[col] = None

merged_df = merged_df[ordered_columns]

# Guardamos el archivo final
final_output_path = '/content/restaurantes_final.csv'
merged_df.to_csv(final_output_path, index=False)

print(f"\nArchivo final procesado y guardado en: {final_output_path}")




Claves únicas en combined_df:
   id_ciudad           ciudad codigo_postal_ciudad
0       20si           apopka                32811
2       w2w7          brandon                33605
4       at0h  west palm beach                33436
5       20si           apopka                32801
6       20si           apopka                32825
7       w2w7          brandon                33607
8       w2w7          brandon                33602
9       w2w7          brandon                33606
10      w2w7          brandon                33617
12      w2w7          brandon                33620

Claves únicas en ciudades_dim:
  id_ciudad             ciudad codigo_postal_ciudad
0      rkx1           aberdeen                33472
1      ikse            alachua                32601
2      ikse            alachua                32605
3      ikse            alachua                32608
4      jccr  alexander springs                32702
5      1ck3             alford                32420
6      c6xs 

Completamos los datos de ciudad

In [ ]:
#  Cargamos los archivos
restaurantes_file = '/content/restaurantes_final.csv'
ciudades_dim_file = '/content/sample_data/ciudades_dim.csv'

restaurantes_df = pd.read_csv(restaurantes_file)
ciudades_dim_df = pd.read_csv(ciudades_dim_file)

# Normalizamos las  claves en ambos DataFrames
restaurantes_df['ciudad'] = restaurantes_df['ciudad'].astype(str).str.strip().str.lower()
ciudades_dim_df['ciudad'] = ciudades_dim_df['ciudad'].astype(str).str.strip().str.lower()

# Realizamos el merge
merged_df = restaurantes_df.merge(
    ciudades_dim_df[['id_condado', 'condado', 'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'ciudad']],
    on='ciudad',
    how='left'
)

# Eliminamos columnas duplicadas
merged_df = merged_df.drop(columns=['id_condado_x', 'condado_x', 'codigo_postal_condado_x', 'latitud_condado_x', 'longitud_condado_x'], errors='ignore')

# Renombramos columnas terminadas en '_y'
merged_df.rename(columns={
    'id_condado_y': 'id_condado',
    'condado_y': 'condado',
    'codigo_postal_condado_y': 'codigo_postal_condado',
    'latitud_condado_y': 'latitud_condado',
    'longitud_condado_y': 'longitud_condado'
}, inplace=True)

# Guardamos el archivo actualizado
output_path = '/content/restaurantes_final_actualizado.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo final actualizado guardado en: {output_path}")



Archivo final actualizado guardado en: /content/restaurantes_final_actualizado.csv


reordenamos las columnas

In [ ]:
#Cargamos la ruta del archivo
input_path = '/content/restaurantes_final_actualizado.csv'
merged_df = pd.read_csv(input_path)

# Definir el orden de las columnas deseado
ordered_columns = [
    'id_servicios_restaurantes', 'servicios_restaurantes', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad', 'codigo_postal_ciudad',
    'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio', 'enlaces_google_maps', 'anio'
]


for col in ordered_columns:
    if col not in merged_df.columns:
        merged_df[col] = None


merged_df = merged_df[ordered_columns]

# Guardamos el archivo ordenado
output_path = '/content/restaurantes_final_ordenado.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo con columnas ordenadas guardado en: {output_path}")




Archivo con columnas ordenadas guardado en: /content/restaurantes_final_ordenado.csv


Canbiamos los nombres de clolumnas

In [ ]:
# Cargamos los archivos
file1 = '/content/drive/MyDrive/categorias10/restaurantes_actualizado.csv'
file2 = '/content/drive/MyDrive/categorias3.3/restaurantes_ordenado.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Renombramos las columnas en ambos DataFrames
rename_mapping = {
    'id_centros_educacion': 'id_servicios_restaurantes',
    'centros_restaurantes': 'servicios_restaurantes'
}

df1.rename(columns=rename_mapping, inplace=True)
df2.rename(columns=rename_mapping, inplace=True)

# Guardamos los archivos actualizados
file1_output = '/content/drive/MyDrive/categorias10/restaurantes_actualizado_renombrado.csv'
file2_output = '/content/drive/MyDrive/categorias3.3/restaurantes_ordenado_renombrado.csv'

df1.to_csv(file1_output, index=False)
df2.to_csv(file2_output, index=False)

file1_output, file2_output


('/content/drive/MyDrive/categorias10/restaurantes_actualizado_renombrado.csv',
 '/content/drive/MyDrive/categorias3.3/restaurantes_ordenado_renombrado.csv')

Unificamos los nombres en las columnas

In [ ]:
#Cargamos la ruta de entrada y salida
input_file = '/content/drive/MyDrive/categorias10/restaurantes_actualizado_renombrado.csv'
output_file = '/content/drive/MyDrive/categorias10/restaurantes_actualizado_renombrado_corrigido.csv'

df = pd.read_csv(input_file)

# Renombramos las columnas
column_renames = {
    'atributo': 'atributos',
    'categoria': 'categorias',
    '.url': 'url_del_negocio',
    'ranquing_por_usuario': 'puntuacion_usuarios',
    'id_centros_restaurantes':'id_servicios_restaurantes'
}

df.rename(columns=column_renames, inplace=True)

# Guardamos el archivo actualizado
df.to_csv(output_file, index=False)

print(f"Archivo actualizado guardado en: {output_file}")


Archivo actualizado guardado en: /content/drive/MyDrive/categorias10/restaurantes_actualizado_renombrado_corrigido.csv


Unificamos los archivos obtenidos  de Google Yelp y Overpass

In [ ]:
# Cargamos los archivos necesarios para el merge

file1 = '/content/restaurantes_final_actualizado.csv'
file2 = '/content/drive/MyDrive/categorias10/restaurantes_actualizado_renombrado_corrigido.csv'
file3 = '/content/drive/MyDrive/categorias3.3/restaurantes_ordenado_renombrado.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

# Realizamos el merge de los tres archivos
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

required_columns = [
    'id_servicios_restaurantes', 'servicios_restaurantes', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad', 'codigo_postal_ciudad',
    'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos', 'categorias', 'puntuacion_usuarios',
    'numero_de_reviews', 'analisis_sentimientos', 'url_del_negocio', 'enlaces_google_maps', 'anio'
]

for col in required_columns:
    if col not in merged_df.columns:
        merged_df[col] = None

# Ordenamos las columnas
merged_df = merged_df[required_columns]

# Eliminamos duplicados
merged_df.drop_duplicates(inplace=True)

# Guardamos el resultado final
output_path = '/content/sample_data/servicio_de_restaurant_ordenado.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo combinado, sin duplicados y con columnas ordenadas guardado en: {output_path}")


Archivo combinado, sin duplicados y con columnas ordenadas guardado en: /content/sample_data/servicio_de_restaurant_ordenado.csv


Eliminamos caracteres inesesarios

In [ ]:
#Cargamos el archivo a prosesar
input_file = '/content/sample_data/servicio_de_restaurant_ordenado.csv'
output_file = '/content/sample_data/servicio_de_restaurant_ordenado2.csv'

df = pd.read_csv(input_file, dtype={'id_centros_entretenimiento': 'string'}, low_memory=False)

# Normalizamos los nombres de las columnas
df.columns = [
    col_name.strip()
    .replace(" ", "_")
    .replace(",", "")
    .replace("(", "")
    .replace(")", "")
    .replace("$", "")
    .replace(";", "")
    .replace("=", "")
    .replace(".", "_")
    .replace("\n", "")
    .replace("\t", "")
    for col_name in df.columns
]

# Asignar el formatos a las columnas
column_types = {
    'id_centros_entretenimiento': 'string',
    'centros_entretenimiento': 'string',
    'direccion': 'string',
    'id_condado': 'string',
    'condado': 'string',
    'codigo_postal_condado': 'string',
    'latitud_condado': 'float',
    'longitud_condado': 'float',
    'id_ciudad': 'string',
    'ciudad': 'string',
    'codigo_postal_ciudad': 'string',
    'latitud_ciudad': 'float',
    'longitud_ciudad': 'float',
    'estado': 'string',
    'categorias': 'string',
    'puntuacion_usuarios': 'float',
    'analisis_sentimientos': 'string',
    'url_del_negocio': 'string',
    'enlaces_google_maps': 'string',
    'anio': 'int'
}

for column, dtype in column_types.items():
    if column in df.columns:
        df[column] = df[column].astype(dtype)

# Eliminamos caracteres no deseados en todas las columnas de tipo string
columns_to_clean = df.select_dtypes(include=['string']).columns
df[columns_to_clean] = df[columns_to_clean].replace(
    to_replace=r"[\[\]\{\}'\"]", value="", regex=True
)

# Guardamos el archivo procesado
df.to_csv(output_file, index=False)

print(f"Archivo procesado y guardado en: {output_file}")


Archivo procesado y guardado en: /content/sample_data/servicio_de_restaurant_ordenado2.csv


Unificamos los nombres delos archivos

In [ ]:
# Cargamos los archivos de entrada y salida
input_file = '/content/drive/MyDrive/categorias10/servicios_hosteleria.csv'
output_file = '/content/drive/MyDrive/categorias10/servicios_hosteleria_coregido.csv'

df = pd.read_csv(input_file)

# Renombramos las columnas
column_renames = {
    'atributo': 'atributos',
    'categoria': 'categorias',
    '.url': 'url_del_negocio',
    'ranquing_por_usuario': 'puntuacion_usuarios'

}

df.rename(columns=column_renames, inplace=True)

# Guardamos el archivo actualizado
df.to_csv(output_file, index=False)

print(f"Archivo actualizado guardado en: {output_file}")

Archivo actualizado guardado en: /content/drive/MyDrive/categorias10/servicios_hosteleria_coregido.csv


Unificamos los archivos obtenidos  de Google  y Yelp

In [ ]:
#Cargamos los archivos de entrada y salida
file1 = '/content/drive/MyDrive/categorias10/servicios_hosteleria_coregido.csv'
file2 = '/content/drive/MyDrive/categorias3.3/hosteleria_ordenado.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Realizamos el merge
merged_df = pd.concat([df1, df2], ignore_index=True)

required_columns = [
    'id_centros_hosteleria', 'centros_hosteleria', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad',
    'codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos',
    'categorias', 'puntuacion_usuarios', 'numero_de_reviews', 'analisis_sentimientos',
    'url_del_negocio', 'enlaces_google_maps', 'anio'
]

for col in required_columns:
    if col not in merged_df.columns:
        merged_df[col] = None

# Ordenamos las columnas
merged_df = merged_df[required_columns]

# Eliminamos duplicados
merged_df['non_null_count'] = merged_df.notna().sum(axis=1)
merged_df = merged_df.sort_values(by='non_null_count', ascending=False)
merged_df = merged_df.drop_duplicates(subset=['id_centros_hosteleria', 'anio'], keep='first')

# Ordenamos los datos por la columna 'anio' de menor a mayor
merged_df = merged_df.sort_values(by='anio', ascending=True).drop(columns=['non_null_count'], errors='ignore')

# Guardamos los resultados
output_path = '/content/sample_data/servicio_hospedaje.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo combinado, sin duplicados y con columnas preservadas guardado en: {output_path}")


Archivo combinado, sin duplicados y con columnas preservadas guardado en: /content/sample_data/servicio_hospedaje.csv


Eliminamos caracteres inesesarios y damos un formato unificado a las columnas

In [ ]:
# Cargamos los archivos de entrada y salida
input_file = '/content/sample_data/servicio_hospedaje.csv'
output_file = '/content/sample_data/servicio_hospedaje2.csv'

df = pd.read_csv(input_file)

# Normalizamos nombres de columnas
df.columns = [
    col_name.strip()
    .replace(" ", "_")
    .replace(",", "")
    .replace("(", "")
    .replace(")", "")
    .replace("$", "")
    .replace(";", "")
    .replace("=", "")
    .replace(".", "_")
    .replace("\n", "")
    .replace("\t", "")
    for col_name in df.columns
]

# Asignamos el tipo de datos a las columnas
column_types = {
    'id_centros_hosteleria': 'string',
    'centros_hosteleria': 'string',
    'direccion': 'string',
    'id_condado': 'string',
    'condado': 'string',
    'codigo_postal_condado': 'string',
    'latitud_condado': 'float',
    'longitud_condado': 'float',
    'id_ciudad': 'string',
    'ciudad': 'string',
    'codigo_postal_ciudad': 'string',
    'latitud_ciudad': 'float',
    'longitud_ciudad': 'float',
    'estado': 'string',
    'categorias': 'string',
    'puntuacion_usuarios': 'float',
    'analisis_sentimientos': 'string',
    'url_del_negocio': 'string',
    'enlaces_google_maps': 'string',
    'anio': 'int'
}

for column, dtype in column_types.items():
    if column in df.columns:
        df[column] = df[column].astype(dtype)

# Eliminamos caracteres no deseados en todas las columnas de tipo string
columns_to_clean = df.select_dtypes(include=['string']).columns
df[columns_to_clean] = df[columns_to_clean].replace(
    to_replace=r"[\[\]\{\}'\"]", value="", regex=True
)


# Guardamos el archivo prosesado
df.to_csv(output_file, index=False)

print(f"Archivo procesado y guardado en: {output_file}")

Archivo procesado y guardado en: /content/sample_data/servicio_hospedaje2.csv


Renombramos columnas para poder realisar el merge

In [ ]:
# Cargamos los archivos de entrada y salida
input_file = '/content/drive/MyDrive/categorias10/tiendas_negocio_actualizado.csv'
output_file = '/content/drive/MyDrive/categorias10/tiendas_negocio_actualizado_coregido.csv'

df = pd.read_csv(input_file)

# Renombramos las columnas
column_renames = {
    'atributo': 'atributos',
    'categoria': 'categorias',
    '.url': 'url_del_negocio',
    'ranquing_por_usuario': 'puntuacion_usuarios'

}

df.rename(columns=column_renames, inplace=True)

# Guardamos el archivo actualizado
df.to_csv(output_file, index=False)

print(f"Archivo actualizado guardado en: {output_file}")

Archivo actualizado guardado en: /content/drive/MyDrive/categorias10/tiendas_negocio_actualizado_coregido.csv


Unificamos los archivos obtenidos  de Google y Yelp

In [ ]:
#Cargamos los archivos nesesarios para el merge
file1 = '/content/drive/MyDrive/categorias10/tiendas_negocio_actualizado_coregido.csv'
file2 = '/content/drive/MyDrive/categorias3.3/servicio_tiendas_negocio_ordenado.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

#Realizamos el merge de los archivos, preservando todas las columnas
merged_df = pd.concat([df1, df2], ignore_index=True)

required_columns = [
    'id_centros_tiendas_negocio', 'centros_tiendas_negocio', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad',
    'codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos',
    'categorias', 'puntuacion_usuarios', 'numero_de_reviews', 'analisis_sentimientos',
    'url', 'enlaces_google_maps', 'anio'
]

for col in required_columns:
    if col not in merged_df.columns:
        merged_df[col] = pd.NA

# Reordenamos las columnas
merged_df = merged_df[required_columns]

# Guardamos el resultado
output_path = '/content/servicio_tiendas_negocios_combinedo4.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo combinado guardado en: {output_path}")




Archivo combinado guardado en: /content/servicio_tiendas_negocios_combinedo4.csv


Eliminamos caracteres innesesarios

In [ ]:
# Cargamos los archivos de entrada y salida
input_file = '/content/servicio_tiendas_negocios_combinedo4.csv'
output_file = '/content/servicio_tiendas_negocios_combinedo5.csv'

df = pd.read_csv(input_file)

# Normalizamos las columnas
df.columns = [
    col_name.strip()
    .replace(" ", "_")
    .replace(",", "")
    .replace("(", "")
    .replace(")", "")
    .replace("$", "")
    .replace(";", "")
    .replace("=", "")
    .replace(".", "_")
    .replace("\n", "")
    .replace("\t", "")
    for col_name in df.columns
]

# Asignmos formato a las columnas
column_types = {
    'id_centros_tiendas_negocio': 'string',
    'centros_tiendas_negocio': 'string',
    'direccion': 'string',
    'id_condado': 'string',
    'condado': 'string',
    'codigo_postal_condado': 'string',
    'latitud_condado': 'float',
    'longitud_condado': 'float',
    'id_ciudad': 'string',
    'ciudad': 'string',
    'codigo_postal_ciudad': 'string',
    'latitud_ciudad': 'float',
    'longitud_ciudad': 'float',
    'estado': 'string',
    'categorias': 'string',
    'puntuacion_usuarios': 'float',
    'analisis_sentimientos': 'string',
    'url_del_negocio': 'string',
    'enlaces_google_maps': 'string',
    'anio': 'Int64'  }

for column, dtype in column_types.items():
    if column in df.columns:
        try:
            df[column] = df[column].astype(dtype)
        except ValueError as e:
            print(f"Error al convertir la columna {column}: {e}")

# Limpiamos caracteres no deseados en columnas
columns_to_clean = df.select_dtypes(include=['string']).columns
df[columns_to_clean] = df[columns_to_clean].replace(
    to_replace=r"[\[\]\{\}'\"]", value="", regex=True
)

# Guardamos el archivo procesado
df.to_csv(output_file, index=False)
print(f"Archivo procesado y guardado en: {output_file}")





Archivo procesado y guardado en: /content/servicio_tiendas_negocios_combinedo5.csv


Unificamos nombres en los archivos

In [ ]:
# Cargamos los archivos de entrada y salida
input_file = '/content/drive/MyDrive/categorias10/transporte_actualizado.csv'
output_file = '/content/drive/MyDrive/categorias10/transporte_actualizado_coregido.csv'

df = pd.read_csv(input_file)

# Renombramos las columnas
column_renames = {
    'atributo': 'atributos',
    'categoria': 'categorias',
    '.url': 'url_del_negocio',
    'ranquing_por_usuario': 'puntuacion_usuarios'

}

df.rename(columns=column_renames, inplace=True)

# Guardamos el archivo actualizado
df.to_csv(output_file, index=False)

print(f"Archivo actualizado guardado en: {output_file}")

Archivo actualizado guardado en: /content/drive/MyDrive/categorias10/transporte_actualizado_coregido.csv


In [ ]:
#cargamos los archivos a comcatenar
file1 = '/content/drive/MyDrive/categorias10/transporte_actualizado.csv'
file2 = '/content/drive/MyDrive/categorias3.3/servicio_transporte_ordenado.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

df1.rename(columns={'id_transporte': 'id_trasporte', 'transporte': 'trasporte'}, inplace=True)
df2.rename(columns={'id_transporte': 'id_trasporte', 'transporte': 'trasporte'}, inplace=True)

# Concatenamos los archivos
merged_df = pd.concat([df1, df2], ignore_index=True)

required_columns = [
    'id_trasporte', 'trasporte', 'direccion', 'id_condado', 'condado',
    'codigo_postal_condado', 'latitud_condado', 'longitud_condado', 'id_ciudad', 'ciudad',
    'codigo_postal_ciudad', 'latitud_ciudad', 'longitud_ciudad', 'estado', 'atributos',
    'categorias', 'puntuacion_usuarios', 'numero_de_reviews', 'analisis_sentimientos',
    'url', 'enlaces_google_maps', 'anio'
]

for col in required_columns:
    if col not in merged_df.columns:
        merged_df[col] = pd.NA


merged_df = merged_df[required_columns]

output_path = '/content/sample_data/servicio_transporte_ordenado2.csv'
merged_df.to_csv(output_path, index=False)

print(f"Archivo combinado guardado en: {output_path}")



Archivo combinado guardado en: /content/sample_data/servicio_transporte_ordenado2.csv


Eliminamos caracteres innesesarios

In [ ]:
#Cargamos las rutas delos archivos
input_file = '/content/sample_data/servicio_transporte_ordenado2.csv'
output_file = '/content/sample_data/servicio_transporte_ordenado3.csv'

if not os.path.exists(input_file):
    print(f"El archivo no se encontró: {input_file}")
    exit()

df = pd.read_csv(input_file)

# Normalizamos los nombres de columnas
df.columns = [
    col_name.strip()
    .replace(" ", "_")
    .replace(",", "")
    .replace("(", "")
    .replace(")", "")
    .replace("$", "")
    .replace(";", "")
    .replace("=", "")
    .replace(".", "_")
    .replace("\n", "")
    .replace("\t", "")
    for col_name in df.columns
]

# Asignamos el formato correspondiente a cada columna
column_types = {
    'id_centros_tiendas_negocio': 'string',
    'centros_tiendas_negocio': 'string',
    'direccion': 'string',
    'id_condado': 'string',
    'condado': 'string',
    'codigo_postal_condado': 'string',
    'latitud_condado': 'float',
    'longitud_condado': 'float',
    'id_ciudad': 'string',
    'ciudad': 'string',
    'codigo_postal_ciudad': 'string',
    'latitud_ciudad': 'float',
    'longitud_ciudad': 'float',
    'estado': 'string',
    'categorias': 'string',
    'puntuacion_usuarios': 'float',
    'analisis_sentimientos': 'string',
    'url_del_negocio': 'string',
    'enlaces_google_maps': 'string',
    'anio': 'int'
}

for column, dtype in column_types.items():
    if column in df.columns:
        df[column] = df[column].astype(dtype)

# Limpiamos los caracteres no deseados en columnas
columns_to_clean = df.select_dtypes(include=['string']).columns
df[columns_to_clean] = df[columns_to_clean].replace(
    to_replace=r"[\[\]\{\}'\"]", value="", regex=True
)

# Guardamos el archivo procesado
df.to_csv(output_file, index=False)
print(f"Archivo procesado y guardado en: {output_file}")


Archivo procesado y guardado en: /content/sample_data/servicio_transporte_ordenado3.csv
